In [10]:
import numpy as np
import pandas as pd

In [11]:
X_tx_matrix = np.load("X_tx_matrix.npy")
y = np.load("y_fico_scores.npy")

In [12]:
X_tx_matrix = np.nan_to_num(X_tx_matrix, nan=0.0, posinf=1e6, neginf=-1e6)
y = np.nan_to_num(y, nan=0.0)

In [13]:
print("Feature variances:", X_tx_matrix.var(axis=(0, 1)))

Feature variances: 2.1745524e+18


In [14]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}")

# Parameters
num_epochs = 10
batch_size = 16

# Load data
print("📥 Loading data...")
X_tx_matrix = np.load("X_tx_matrix.npy")
y = np.load("y_fico_scores.npy")
print(f"✅ X shape: {X_tx_matrix.shape}, y shape: {y.shape}")

# Clean and scale
print("🧼 Cleaning data...")
X_tx_matrix = np.nan_to_num(X_tx_matrix, nan=0.0, posinf=1e6, neginf=-1e6)
y = np.nan_to_num(y, nan=0.0)

print(f"NaNs remaining in X: {np.isnan(X_tx_matrix).sum()}")
print(f"Feature variances before scaling: {X_tx_matrix.var(axis=(0, 1))}")

scaler = StandardScaler()
flat = X_tx_matrix.reshape(-1, X_tx_matrix.shape[-1])
flat_scaled = scaler.fit_transform(flat)
X_tx_matrix_scaled = flat_scaled.reshape(X_tx_matrix.shape)

print(f"Feature variances after scaling: {X_tx_matrix_scaled.var(axis=(0, 1))}")

# Convert to torch tensors
X_tensor = torch.tensor(X_tx_matrix_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

print("X_tensor shape:", X_tensor.shape)
print("y_tensor shape:", y_tensor.shape)

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Simple MLP model
class MLPFICO(nn.Module):
    def __init__(self, input_dim=4, seq_len=100):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim * seq_len, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x).squeeze(-1)

# Model, loss, optimizer
model = MLPFICO().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
criterion = nn.MSELoss()

# Training
print("\n🏋️ Starting training...")
for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    for batch_i, (batch_x, batch_y) in enumerate(dataloader):
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        preds = model(batch_x)

        if torch.isnan(preds).any():
            print("❌ NaN detected in predictions — check inputs!")
            break

        loss = criterion(preds, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

        if batch_i % 10 == 0:
            print(f"🌀 Epoch {epoch+1}, Batch {batch_i}, Loss: {loss.item():.4f}")

    print(f"✅ Epoch {epoch+1} finished. Avg Loss: {np.mean(epoch_losses):.4f}")

# Evaluation
print("\n🧪 Evaluating model...")
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for batch_x, batch_y in dataloader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        preds = model(batch_x)

        if torch.isnan(preds).any():
            print("❌ NaNs detected in evaluation predictions")
            break

        all_preds.append(preds.cpu().numpy())
        all_targets.append(batch_y.cpu().numpy())

# Flatten arrays
all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)

print(f"📏 Predictions: mean={all_preds.mean():.2f}, std={all_preds.std():.2f}")
print(f"🎯 Targets:     mean={all_targets.mean():.2f}, std={all_targets.std():.2f}")

# Calculate metrics
mae = mean_absolute_error(all_targets, all_preds)
r2 = r2_score(all_targets, all_preds)

print("\n📊 Evaluation Metrics:")
print(f"MAE:  {mae:.2f}")
print(f"R²:   {r2:.3f}")

# Prediction samples
print("\n🔍 Sample Predictions:")
for i in range(10):
    print(f"True: {all_targets[i]:.1f}, Predicted: {all_preds[i]:.1f}")

🖥️ Using device: cpu
📥 Loading data...
✅ X shape: (100, 4), y shape: (1,)
🧼 Cleaning data...
NaNs remaining in X: 0
Feature variances before scaling: 2.174552387325264e+18
Feature variances after scaling: 1.0
X_tensor shape: torch.Size([100, 4])
y_tensor shape: torch.Size([1])


AssertionError: Size mismatch between tensors

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge  # LinearRegression if you don't want L2
from sklearn.metrics import mean_absolute_error, r2_score

# Load data
X_tx = np.load("sim_data/X_tx_matrix.npy")              # (N, 100, 4)
X_wallet = np.load("sim_data/X_wallet_features.npy")    # (N, 5)
y = np.load("sim_data/y_fico_scores.npy")               # (N,)

# Clean NaNs/Infs
X_tx = np.nan_to_num(X_tx, nan=0.0, posinf=1e6, neginf=-1e6)
X_wallet = np.nan_to_num(X_wallet, nan=0.0, posinf=1e6, neginf=-1e6)
y = np.nan_to_num(y, nan=0.0)

# Feature engineering
# Pool transaction features into simple summary stats
X_tx_mean = X_tx.mean(axis=1)     # (N, 4)
X_tx_std = X_tx.std(axis=1)       # (N, 4)

X_combined = np.concatenate([X_tx_mean, X_tx_std, X_wallet], axis=1)  # (N, 13)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)

# Train/test split (optional — full train for now)
model = Ridge(alpha=1.0)  # Use LinearRegression() if you want no regularization
model.fit(X_scaled, y)

# Predict
y_pred = model.predict(X_scaled)

# Eval
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("\n📊 Simple Model Metrics:")
print(f"MAE: {mae:.2f}")
print(f"R²:  {r2:.3f}")

print("\n🔍 Sample Predictions:")
for i in range(10):
    print(f"True: {y[i]:.1f}, Pred: {y_pred[i]:.1f}")



📊 Simple Model Metrics:
MAE: 15.29
R²:  0.924

🔍 Sample Predictions:
True: 350.0, Pred: 316.9
True: 500.0, Pred: 491.0
True: 543.0, Pred: 538.5
True: 489.0, Pred: 533.3
True: 515.0, Pred: 516.0
True: 545.0, Pred: 548.6
True: 589.0, Pred: 626.6
True: 730.0, Pred: 661.7
True: 642.0, Pred: 627.6
True: 419.0, Pred: 421.4


In [18]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

# Load data
X_tx = np.load("sim_data/X_tx_matrix.npy")              # shape: (N, 100, 4)
X_wallet = np.load("sim_data/X_wallet_features.npy")    # shape: (N, 5)
y = np.load("sim_data/y_fico_scores.npy")               # shape: (N,)

# Clean invalid values
X_tx = np.nan_to_num(X_tx, nan=0.0, posinf=1e6, neginf=-1e6)
X_wallet = np.nan_to_num(X_wallet, nan=0.0, posinf=1e6, neginf=-1e6)
y = np.nan_to_num(y, nan=0.0)

# Feature engineering: mean & std over transaction sequences
X_tx_mean = X_tx.mean(axis=1)  # shape: (N, 4)
X_tx_std = X_tx.std(axis=1)    # shape: (N, 4)

# Combine all features
X_combined = np.concatenate([X_tx_mean, X_tx_std, X_wallet], axis=1)  # shape: (N, 13)

# Standardize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)

# Define and train XGBoost model
final_model = XGBRegressor(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    objective="reg:squarederror",
    verbosity=1,
    n_jobs=-1
)
final_model.fit(X_scaled, y)

# Predict and evaluate
y_pred = final_model.predict(X_scaled)

mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("\n📊 XGBoost Model Evaluation:")
print(f"MAE: {mae:.2f}")
print(f"R²:  {r2:.3f}")

print("\n🔍 Sample Predictions:")
for i in range(10):
    print(f"True: {y[i]:.1f}, Pred: {y_pred[i]:.1f}")


📊 XGBoost Model Evaluation:
MAE: 7.34
R²:  0.985

🔍 Sample Predictions:
True: 350.0, Pred: 349.8
True: 500.0, Pred: 512.4
True: 543.0, Pred: 553.5
True: 489.0, Pred: 483.9
True: 515.0, Pred: 525.7
True: 545.0, Pred: 555.9
True: 589.0, Pred: 586.6
True: 730.0, Pred: 723.4
True: 642.0, Pred: 647.3
True: 419.0, Pred: 433.0


In [19]:
with open("fico_xgb_model.pkl", "wb") as f_model:
    pickle.dump(final_model, f_model)

# Save scaler used on X_combined
with open("fico_xgb_scaler.pkl", "wb") as f_scaler:
    pickle.dump(scaler, f_scaler)

print("💾 Saved: fico_xgb_model.pkl and fico_xgb_scaler.pkl")


💾 Saved: fico_xgb_model.pkl and fico_xgb_scaler.pkl
